In [1]:
import os
from typing import List, Dict, Any
from pinecone import Pinecone
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.prompts import PromptTemplate

In [ ]:
pinecone_api = ""
groq_api = ""
groq_llm_model = "llama-3.3-70b-versatile"
huggingface_embeddings_model = "sentence-transformers/all-MiniLM-L6-v2"
pinecone_index_name = "stock-data-index-5"

In [11]:
def initialize_rag_components(pinecone_api: str, 
                              groq_api: str, 
                              groq_llm_model: str, 
                              huggingface_embeddings_model: str,
                              pinecone_index_name: str) -> tuple:
    """Initialize all RAG components: Pinecone, Groq LLM, and HuggingFace embeddings"""
    
    # Initialize Pinecone
    pc = Pinecone(api_key=pinecone_api)
    index = pc.Index(pinecone_index_name)

    # Initialize Groq LLM
    llm = ChatGroq(
        groq_api_key=groq_api,
        model=groq_llm_model,
        temperature=0.1,
        max_tokens=1000
    )
    
    # Initialize HuggingFace embeddings (same as used in indexing)
    embeddings = HuggingFaceEmbeddings(
        model_name=huggingface_embeddings_model,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    
    return index, llm, embeddings

In [12]:
def retrieve_relevant_context(query: str, index, embeddings, top_k: int = 5) -> List[Dict]:
    """Retrieve relevant stock data from Pinecone"""
    
    try:
        # Convert query to embedding
        query_embedding = embeddings.embed_query(query)
        
        # Search Pinecone for similar vectors
        search_results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True,
            include_values=False
        )
        
        # Extract relevant documents
        retrieved_docs = []
        for match in search_results.matches:
            retrieved_docs.append({
                'score': match.score,
                'content': match.metadata.get('content', 'Content not available'),
                'metadata': {
                    'ticker': match.metadata.get('ticker'),
                    'company_name': match.metadata.get('company_name'),
                    'sector': match.metadata.get('sector'),
                    'industry': match.metadata.get('industry'),
                    'update_date': match.metadata.get('update_date')
                },
                'id': match.id
            })
        
        return retrieved_docs
        
    except Exception as e:
        print(f"Error retrieving context: {e}")
        return []

In [13]:
def format_context_for_llm(retrieved_docs: List[Dict]) -> str:
    """Format retrieved documents for LLM context"""
    
    if not retrieved_docs:
        return "No relevant stock data found."
    
    context_parts = []
    
    for i, doc in enumerate(retrieved_docs, 1):
        context_parts.append(f"=== Stock Data {i} (Relevance: {doc['score']:.3f}) ===")
        context_parts.append(doc['content'])  # This contains all the formatted stock info
        context_parts.append("")  # Spacing
    
    return "\n".join(context_parts)

In [14]:
def create_stock_prompt_template():
    """Create a prompt template for stock-related queries"""
    
    template = """
You are a knowledgeable financial advisor assistant. Based on the stock data provided below, answer the user's question accurately and concisely.

CONTEXT (Retrieved Stock Data):
{context}

USER QUESTION: {question}

INSTRUCTIONS:
1. Answer based ONLY on the provided stock data
2. If the information isn't available in the context, clearly state that
3. Provide specific numbers when available (percentages, dollar amounts, etc.)
4. If multiple stocks are relevant, compare them
5. Be conversational but professional

ANSWER:
"""
    
    return PromptTemplate(
        template=template,
        input_variables=["context", "question"]
    )

In [17]:
def rag_query_stocks(query: str, top_k: int = 5) -> Dict[str, Any]:
    """Enhanced RAG function using stored page content"""
    
    try:
        print(f"Query: {query}")
        
        # Initialize components
        index, llm, embeddings = initialize_rag_components(
            pinecone_api=pinecone_api,
            groq_api=groq_api,
            groq_llm_model=groq_llm_model,
            huggingface_embeddings_model=huggingface_embeddings_model,
            pinecone_index_name=pinecone_index_name
        )
        
        # Use enhanced retrieval with stored content
        print("Retrieving relevant stock data with full content...")
        retrieved_docs = retrieve_relevant_context(query, index, embeddings, top_k)
        
        if not retrieved_docs:
            return {
                "answer": "I couldn't find any relevant stock data for your query. Please check if the stock is in our database.",
                "context": [],
                "query": query,
                "success": False
            }
        
        # Format context using full content
        context_text = format_context_for_llm(retrieved_docs)
        
        # Create prompt
        prompt_template = create_stock_prompt_template()
        formatted_prompt = prompt_template.format(
            context=context_text,
            question=query
        )
        
        # Generate answer using Groq
        print("Generating answer with Groq...")
        response = llm.invoke(formatted_prompt)
        answer = response.content
        
        return {
            "answer": answer,
            "context": retrieved_docs,
            "query": query,
            "num_sources": len(retrieved_docs),
            "success": True
        }
        
    except Exception as e:
        return {
            "answer": f"An error occurred while processing your query: {str(e)}",
            "context": [],
            "query": query,
            "success": False
        }

In [18]:
# Example usage functions
def main_examples():
    """Example usage of the RAG functions"""
    
    # Example 1: Basic query
    print("=== Example 1: Basic Query ===")
    result1 = rag_query_stocks("What is the annualized return of Apple?")
    print(f"Answer: {result1['answer']}")
    print(f"Sources: {result1['num_sources']}")
    print()
    
    # Example 2: Comparison query
    print("=== Example 2: Comparison Query ===")
    result2 = rag_query_stocks("What is the annualized return of NVIDIA?")
    print(f"Answer: {result2['answer']}")
    print()

if __name__ == "__main__":
    main_examples()

=== Example 1: Basic Query ===
Query: What is the annualized return of Apple?
Retrieving relevant stock data with full content...
Generating answer with Groq...
Answer: The annualized return of Apple (AAPL) is 20.76%. This is based on the provided stock data for Apple Inc. If you'd like to compare this to other stocks, the closest annualized return is ServiceNow (NOW) at 21.4% and IBM at 22.06%, while the other stocks have lower annualized returns.
Sources: 5

=== Example 2: Comparison Query ===
Query: What is the annualized return of NVIDIA?
Retrieving relevant stock data with full content...
Generating answer with Groq...
Answer: The annualized return of NVIDIA (NVDA) is 72.79%. This is significantly higher than the other stocks listed, with the next closest being Netflix (NFLX) at 23.17%. If you're looking for a comparison, NVIDIA's annualized return is roughly three times that of Netflix, indicating a much stronger historical performance.

